In [ ]:
# Run the following cell if using Google Colab

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/cs340/project/models

!pip install git+https://github.com/openai/CLIP.git

In [1]:
import clip_feature_extractor
import numpy as np

# from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

device = torch.device("cuda")
print(device)

import cupy as cp
import cudf

%load_ext autoreload
%autoreload 2

cuda


In [2]:
X_train_CIFAR100_np, y_train_CIFAR100_np, X_test_CIFAR100_np, y_test_CIFAR100_np = clip_feature_extractor.get_CIFAR100_features();

# TODO this doesn't help? Or redundant with batchnorm? Or is it still needed with batchnorm?
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Ensure your data is in NumPy array format or similar.
X_train = torch.tensor(X_train_CIFAR100_np, dtype=torch.float32)
y_train = torch.tensor(y_train_CIFAR100_np, dtype=torch.long)
X_test  = torch.tensor(X_test_CIFAR100_np, dtype=torch.float32)
y_test  = torch.tensor(y_test_CIFAR100_np, dtype=torch.long)

full_train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified
Extracting features from CIFAR100 dataset
Loaded previously extracted features from disk.


In [53]:
# Can overfit, 97% 50 epochs
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 2048)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(2048, 1024)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(1024, 512)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(512, 512)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(512, 512)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(512, 256)
        self.relu6 = nn.ReLU()
        self.fc7 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.relu5(x)
        x = self.fc6(x)
        x = self.relu6(x)
        x = self.fc7(x)
        
        return x

In [13]:
# 97.8% 50 epochs, 97% at 37 epochs, trains faster?
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(1024, 1024)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(1024, 512)
        self.relu3 = nn.ReLU()
        
        self.fc4 = nn.Linear(512, 256)
        self.relu4 = nn.ReLU()
        
        self.fc5 = nn.Linear(256, 256)
        self.relu5 = nn.ReLU()
        
        self.fc6 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        x = self.relu3(x)

        x = self.fc4(x)
        x = self.relu4(x)

        x = self.fc5(x)
        x = self.relu5(x)

        x = self.fc6(x)
        
        return x

In [15]:
# 97.99% 49 epochs, 97% at 37 epochs, trains faster?
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(1024, 1024)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(1024, 512)
        self.relu3 = nn.ReLU()
        
        self.fc4 = nn.Linear(512, 256)
        self.relu4 = nn.ReLU()
        
        self.fc5 = nn.Linear(256, num_classes)
        self.relu5 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        x = self.relu3(x)

        x = self.fc4(x)
        x = self.relu4(x)

        x = self.fc5(x)
        
        return x

In [17]:
# 97.99% 49 epochs, 97% at 37 epochs, trains faster?
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(1024, 1024)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(1024, 512)
        self.relu3 = nn.ReLU()
        
        self.fc4 = nn.Linear(512, num_classes)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        x = self.relu3(x)

        x = self.fc4(x)
        x = self.fc5(x)
        
        return x

In [19]:
# 98.5% 50 epochs, 97% at 19 epochs
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(1024, 1024)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(1024, num_classes)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        return x

In [21]:
# 98.3% 50 epochs, 97% at 22 epochs
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(1024, 512)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(512, num_classes)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        return x

In [23]:
# 98.5% 50 epochs, 97% at 22 epochs
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(256, num_classes)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        return x

In [25]:
# 98.5% 50 epochs, 97% at 22 epochs
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 2048)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(2048, 1024)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(1024, num_classes)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        return x

In [27]:
# 98.3% 50 epochs, 97% at 22 epochs -- This seems sufficient
class fcnet_CIFAR100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(fcnet_CIFAR100, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(1024, 512)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(512, num_classes)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        return x

In [28]:
model = fcnet_CIFAR100(input_size=X_train.shape[1], num_classes=len(set(y_train)))
model.to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 200

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Training 
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total

    # Validation 
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_batch_X, val_batch_y in val_loader:
            val_batch_X, val_batch_y = val_batch_X.to(device), val_batch_y.to(device)
            
            val_outputs = model(val_batch_X)
            v_loss = criterion(val_outputs, val_batch_y)
            
            val_loss += v_loss.item()
            
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_batch_y.size(0)
            val_correct += (val_predicted == val_batch_y).sum().item()
    
    val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy*100:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy*100:.2f}%")

model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0
with torch.no_grad():
    for test_batch_X, test_batch_y in test_loader:
        test_batch_X, test_batch_y = test_batch_X.to(device), test_batch_y.to(device)
        
        test_outputs = model(test_batch_X)
        t_loss = criterion(test_outputs, test_batch_y)
        
        test_loss += t_loss.item()
        
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_total += test_batch_y.size(0)
        test_correct += (test_predicted == test_batch_y).sum().item()

test_loss = test_loss / len(test_loader)
test_accuracy = test_correct / test_total

print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy*100:.2f}%")

Epoch 1/200
Train Loss: 1.4875, Train Accuracy: 60.08%, Val Loss: 0.9641, Val Accuracy: 70.89%
Epoch 2/200


KeyboardInterrupt: 